In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
from sklearn.preprocessing import *

# dist_measure
# 1 = Euclidean Distance
# 2 = Perpendicular Distance
# 3 = Vertical Distance
def find_pips(data: np.array, n_pips: int, dist_measure: int):
    pips_x = [0, len(data) - 1]  # Index
    pips_y = [data[0], data[-1]] # Price

    for curr_point in range(2, n_pips):
        md = 0.0 # Max distance
        md_i = -1 # Max distance index
        insert_index = -1
        # Use a single loop to iterate over all the points
        for i in range(1, len(data) - 1):
            left_adj = bisect.bisect_right(pips_x, i) - 1
            right_adj = left_adj + 1
            # Calculate the distance from the point to the line segment
            d = distance(data, pips_x, pips_y, i, left_adj, right_adj, dist_measure)
            # Update the maximum distance and the insert index
            if d > md:
                md = d
                md_i = i
                insert_index = right_adj
        # Insert the new pip
        pips_x.insert(insert_index, md_i)
        pips_y.insert(insert_index, data[md_i])
    return pips_x, pips_y

# Define a helper function to calculate the distance
def distance(data, pips_x, pips_y, i, left_adj, right_adj, dist_measure):
    time_diff = pips_x[right_adj] - pips_x[left_adj]
    price_diff = pips_y[right_adj] - pips_y[left_adj]
    slope = price_diff / time_diff
    intercept = pips_y[left_adj] - pips_x[left_adj] * slope
    dist_funcs = {
        1: lambda x, y: ( (pips_x[left_adj] - x) ** 2 + (pips_y[left_adj] - y) ** 2 ) ** 0.5 + ( (pips_x[right_adj] - x) ** 2 + (pips_y[right_adj] - y) ** 2 ) ** 0.5, # Euclidean distance
        2: lambda x, y: abs( (slope * x + intercept) - y ) / (slope ** 2 + 1) ** 0.5, # Perpendicular distance
        3: lambda x, y: abs( (slope * x + intercept) - y ) # Vertical distance
    }
    return dist_funcs[dist_measure](i, data[i])

In [ ]:

data = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv", parse_dates=['time'])
data = data.set_index('time')

# take data from 2007 to 2014
data = data['2007-01-01':'2008-01-01']

# add new columns from the time column, year, month, day, hour, minute
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['hour'] = data.index.hour
data['minute'] = data.index.minute
data

In [197]:
n_close_points = 24
n_perc_points = 4
dist_measure = 1
pips_y_list = []
future_candles = 5
# loop through the data
# start from the 24th point
for index in range(n_close_points, len(data)):
    try:
        x = data['close'].iloc[index-n_close_points:index].to_numpy()
        pips_x, pips_y = find_pips(x, n_perc_points, dist_measure)
        scaled_pips_y = StandardScaler().fit_transform(np.array(pips_y).reshape(-1, 1)).reshape(-1)
        pips_y_dict = {f'pip_{i}': scaled_pips_y[i] for i in range(n_perc_points)}
        pips_y_dict['close'] = x[-1]
        pips_y_dict['future_close'] = data['close'].iloc[index+future_candles]
        pips_y_dict['future_return'] = pips_y_dict['future_close'] - pips_y_dict['close']
        pips_y_dict['year'] = data['year'].iloc[index]
        pips_y_dict['month'] = data['month'].iloc[index]
        pips_y_dict['day'] = data['day'].iloc[index]
        pips_y_dict['hour'] = data['hour'].iloc[index]
        pips_y_dict['minute'] = data['minute'].iloc[index]
        # calculate the future drawdown from close to future_close
        # pips_y_dict['future_drawdown'] = pips_y_dict['close'] - pips_y_dict['future_close']
        
        pips_y_list.append(pips_y_dict)
    except:
        print('An exception occurred')
        break
    # print(scaled_pips_y)
    # pd.Series(x).plot()
    # for i in range(n_perc_points):
    #     plt.plot(pips_x[i], pips_y[i], marker='o', color='red')
    # plt.plot(pips_x, pips_y, color='black')
    # plt.show()
# pips_y_df = pd.DataFrame(pips_y_list, columns=[f'pip_{i}' for i in range(n_perc_points)])
# pips_y_df_np = pips_y_df.to_numpy()

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [ ]:
pips_y_list

In [198]:
import pandas as pd
df = pd.DataFrame(pips_y_list)
print(df)

          pip_0     pip_1     pip_2     pip_3    close  future_close  \
0     -1.669675  0.134062  0.680754  0.854860  1.96565       1.96575   
1     -1.660483  0.094141  0.723081  0.843261  1.96545       1.96570   
2     -1.327838 -0.600032  0.867848  1.060022  1.96562       1.96575   
3     -1.684634  0.652694  0.196948  0.834992  1.96543       1.96563   
4     -1.481763 -0.314121  0.682182  1.113702  1.96583       1.96585   
...         ...       ...       ...       ...      ...           ...   
25337 -1.341657 -0.566947  1.123330  0.785275  1.98596       1.98670   
25338 -1.644852  0.123410  1.007541  0.513901  1.98601       1.98905   
25339 -1.389463  0.274353 -0.270813  1.385923  1.98650       1.98940   
25340 -1.357206  0.263901 -0.316681  1.409986  1.98645       1.98876   
25341 -1.370108  0.223041 -0.267649  1.414716  1.98680       1.98690   

       future_return  
0            0.00010  
1            0.00025  
2            0.00013  
3            0.00020  
4            0.00002

In [ ]:
pips_y_df = pd.DataFrame(pips_y_list)
pips_y_df

In [ ]:
# plot the histogram
pips_y_df.hist(bins=100)

In [ ]:
pips_y_df

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d 
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans

np.random.seed(5)

# X = pips_y_df_np
X = pips_y_df_np[:700]

estimators = [
    ("k_means_24", KMeans(n_clusters=24)),
    ("k_means_3", KMeans(n_clusters=3)),
    ("k_means_bad_init", KMeans(n_clusters=3, n_init=1, init="random")),
]

fig = plt.figure(figsize=(13, 10))
titles = ["24 clusters", "3 clusters", "3 clusters, bad init"]
for idx, ((name, est), title) in enumerate(zip(estimators, titles)):
    ax = fig.add_subplot(2, 2, idx + 1, projection="3d", elev=48, azim=134)
    est.fit(X)
    labels = est.labels_
    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(float), edgecolor="k")
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])
    ax.set_xlabel("p0")
    ax.set_ylabel("p1")
    ax.set_zlabel("p2")
    ax.set_title(title)

# plt.subplots_adjust(wspace=0.25, hspace=0.25)
plt.show()

In [ ]:
np.random.seed(5)

est = KMeans(n_clusters=24)
est.fit(pips_y_df_np)
labels = est.labels_


In [ ]:
len(pips_y_df)

In [ ]:
# add a new column namely k_label using labels to pips_y_df

pips_y_df['k_label'] = labels
pips_y_df